# Apprentissage with BertTOPIC

## What is BertTOPIC ?
BERTopic is a topic modeling python library that combines transformer embeddings and clustering model algorithms to identify topics in NLP (Natual Language Processing).

In [120]:
%pip install bertopic

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [121]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import matplotlib.cm as cm

from bertopic import BERTopic

In [122]:
from umap import UMAP

In [123]:
%pip install langdetect

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [124]:
from langdetect import detect

In [125]:
import ast

In [126]:
lexique = {
    "Université": ["campus", "professor", "lecture", "semester", "degree", "enrollment", "tuition", "dormitory", "library", "exam", "grade", "credit", "major", "minor", "class", "lecture hall", "research", "thesis", "dissertation", "plagiarism", "academic integrity", "curriculum", "internship", "fellowship", "grant", "bursary", "tuition fee", "scholarship", "fellowship", "grant", "bursary"],
    "Publicités et Newletters": ["advertising", "promotion", "marketing", "sales", "banner", "brochure", "catalog", "flyer", "poster", "billboard", "commercial", "infomercial", "endorsement", "testimonial", "mailing list", "newsletter", "email marketing", "influencer marketing", "customer acquisition", "conversion rate", "offer", "discount", "product"],
    "Commandes et Tickets": ["order", "purchase", "checkout", "shopping cart", "invoice", "receipt", "ticket", "reservation", "booking", "confirmation", "cancellation", "refund", "delivery", "shipping", "tracking", "customer service", "call center", "hotline", "escalation", "complaint", "feedback", "review", "rating", "warranty", "guarantee", "return policy", "exchange policy"],
    "Banque et Factures": ["bank", "savings account", "checking account", "credit card", "debit card", "ATM", "online banking", "mobile banking", "investment", "asset", "liability", "interest", "fees", "charge", "balance", "transaction", "deposit", "withdrawal", "transfer", "loan", "mortgage", "insurance", "bill", "invoice", "statement", "payment", "late fee", "penalty"],
    "Rendez-vous": ["appointment", "meeting", "monday", "tuesday", "wednesday", "thursday", "friday", "saturday","sunday","schedule", "agenda", "calendar", "deadline", "reminder", "notification", "confirmation", "cancellation", "reschedule", "no-show", "waiting list", "booking", "reservation", "availability", "flexibility", "urgency", "priority", "request", "proposal", "offer", "counteroffer", "negotiation", "agreement", "disagreement", "compromise"],
    "Réseaux Sociaux": ["social media", "network", "platform", "profile", "timeline", "feed", "friend", "follower", "like", "comment", "share", "tag", "hashtag", "mention", "DM", "PM", "post", "tweet", "status", "story", "live", "video", "photo", "image", "link", "hashtag", "trending", "viral", "influencer", "advertising", "marketing"],
    "Administratif": ["administration", "office", "manager", "staff", "employee", "human resources", "personnel", "policy", "procedure", "regulation", "law", "contract", "agreement", "permit", "license", "certificate", "document", "record", "file", "archive", "database", "system", "software", "hardware", "network", "security", "access", "privilege", "permission"],
    "Candidatures job et milieu Professionnel": ["job", "application", "employment", "career", "professional", "work", "resume", "CV", "cover letter", "application", "interview", "selection", "hiring", "promotion", "training", "development", "performance", "evaluation", "feedback", "review", "salary", "benefit", "perk", "incentive", "bonus", "raise", "promotion", "advancement", "progression", "succession"],
    "Comptes et Mots de passe": ["account", "username", "password", "login", "logout", "security", "privacy", "protection", "encryption", "authentication", "verification", "authorization", "access", "permission", "privilege", "restriction", "violation", "hacking", "phishing", "spam", "virus", "malware", "ransomware", "trojan", "worm", "spyware", "adware", "popup"],
    "Logement": ["housing", "rent", "rental", "home", "residence", "apartment", "condo", "townhouse", "mansion", "villa", "manor", "cottage", "farmhouse", "landlord", "tenant", "rent", "lease", "agreement", "contract", "mortgage", "property", "real estate", "development", "construction", "renovation", "furniture", "appliance", "utility", "amenity", "facility"]
}

In [127]:
final_dataset = pd.read_csv("datacleaned/df_10000.CSV")
final_dataset = final_dataset.drop(['Unnamed: 0'], axis=1) #remove columns creted bc of the to_csv() in the cleaning
final_dataset = final_dataset.iloc[0:10000]
print(final_dataset.shape)

(10000, 3)


In [128]:
final_dataset

,objet,corps,adresse
0,"[']', 'mission', 'promotion', 'important', 'me...","['dear', 'student', 'ambassador', 'lutt', 'you...","['etudiants-request', 'utt', 'fr']"
1,"[']', 'message', 'pierre', 'koch', 'personal',...","['english', ']', 'dear', 'fellow', 'student', ...","['etudiants-request', 'utt', 'fr']"
2,"['your', 'instagram', 'password', 'change']","['this', 'confirmation', 'password', 'instagra...","['no-reply', 'mail.instagram', 'com']"
3,"[']', 'fwd', 'challenge', 'inter', 'entreprise...","['de', 'michel', 'legault', 'sport', 'cc', 'sd...","['sport-request', 'utt', 'fr']"
4,"['confirmation', 'ter', 'ticket', 'order']","['confirmation', 'trip', 'monday', 'november',...","['mesbilletsTER-noreply', 'ter-sncf', 'fr']"
...,...,...,...
9995,"['tedxwoman', 'way']","['imagine', 'explore', 'build', 'tedxwoman', '...","['no-reply', 'ted', 'com']"
9996,"[']', 'justification', 'medical', 'absence', '...","['welcome', 'welcome', 'welcome', 'lutt', 'her...","['etudiants-request', 'utt', 'fr']"
9997,"['early', ']', 'good', 'plan', 'velo', 'schola...","['hello', 'the', 'rucher', 'creative', 'offer'...","['vie.etudiante-request', 'utt', 'fr']"
9998,"['week', 'review', 'elon', 'big', 'week']","['techcrunch', 'newsletter', 'saturday', 'apri...","['newsletter', 'techcrunch', 'com']"


In [129]:
from nltk.corpus import stopwords

def stop_words(text):
    stop_words = stopwords.words('french')
    text = [word for word in text if ((word not in stop_words) and (len(word)>1))]
    
    return text

In [130]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [131]:
tt= np.array([])
for i in final_dataset.index:
    texte = ast.literal_eval(final_dataset['corps'][i])
    texte = stop_words(texte)
    final_dataset['corps'][i] = texte
    print(i)
    if ('nan' in texte) or (len(texte) < 4):
        final_dataset.drop(i, axis=0, inplace=True)
    else:
        langue = detect(' '.join(final_dataset['corps'][i]))
        tt = np.append(tt, langue)

final_dataset = final_dataset.reset_index()
final_dataset.head()

Le flux de sortie a été tronqué et ne contient que les 5000 dernières lignes.
5000
5001
5002
5003
5004
5005
5006
5007
5008
5009
5010
5011
5012
5013
5014
5015
5016
5017
5018
5019
5020
5021
5022
5023
5024
5025
5026
5027
5028
5029
5030
5031
5032
5033
5034
5035
5036
5037
5038
5039
5040
5041
5042
5043
5044
5045
5046
5047
5048
5049
5050
5051
5052
5053
5054
5055
5056
5057
5058
5059
5060
5061
5062
5063
5064
5065
5066
5067
5068
5069
5070
5071
5072
5073
5074
5075
5076
5077
5078
5079
5080
5081
5082
5083
5084
5085
5086
5087
5088
5089
5090
5091
5092
5093
5094
5095
5096
5097
5098
5099
5100
5101
5102
5103
5104
5105
5106
5107
5108
5109
5110
5111
5112
5113
5114
5115
5116
5117
5118
5119
5120
5121
5122
5123
5124
5125
5126
5127
5128
5129
5130
5131
5132
5133
5134
5135
5136
5137
5138
5139
5140
5141
5142
5143
5144
5145
5146
5147
5148
5149
5150
5151
5152
5153
5154
5155
5156
5157
5158
5159
5160
5161
5162
5163
5164
5165
5166
5167
5168
5169
5170
5171
5172
5173
5174
5175
5176
5177
5178
5179
5180
5181
5182
5183
51

,index,objet,corps,adresse
0,0,"[']', 'mission', 'promotion', 'important', 'me...","[dear, student, ambassador, lutt, you, make, p...","['etudiants-request', 'utt', 'fr']"
1,1,"[']', 'message', 'pierre', 'koch', 'personal',...","[english, dear, fellow, student, for, four, we...","['etudiants-request', 'utt', 'fr']"
2,2,"['your', 'instagram', 'password', 'change']","[this, confirmation, password, instagram, acco...","['no-reply', 'mail.instagram', 'com']"
3,3,"[']', 'fwd', 'challenge', 'inter', 'entreprise...","[michel, legault, sport, cc, sds, envoy, wedne...","['sport-request', 'utt', 'fr']"
4,4,"['confirmation', 'ter', 'ticket', 'order']","[confirmation, trip, monday, november, hello, ...","['mesbilletsTER-noreply', 'ter-sncf', 'fr']"


In [132]:
uniqus, counts = np.unique(tt, return_counts=True)
dd = dict(zip(uniqus, counts))

dd

{'af': 8,
 'ca': 2,
 'cy': 8,
 'da': 1,
 'de': 25,
 'en': 9050,
 'es': 50,
 'fi': 1,
 'fr': 429,
 'id': 6,
 'it': 29,
 'lt': 2,
 'nl': 23,
 'no': 2,
 'pl': 39,
 'pt': 2,
 'ro': 10,
 'sl': 3,
 'so': 1,
 'sv': 19,
 'tl': 1}

In [133]:
j = 0
cc = 0
while j < len(final_dataset['corps']):
  #if tt[j] == 'en':    
  for mot in final_dataset['corps'][j]:
      if len(mot)<3: 
        cc +=1
        print(cc)
        final_dataset['corps'][j].remove(mot)    
  j=j+1

Le flux de sortie a été tronqué et ne contient que les 5000 dernières lignes.
35449
35450
35451
35452
35453
35454
35455
35456
35457
35458
35459
35460
35461
35462
35463
35464
35465
35466
35467
35468
35469
35470
35471
35472
35473
35474
35475
35476
35477
35478
35479
35480
35481
35482
35483
35484
35485
35486
35487
35488
35489
35490
35491
35492
35493
35494
35495
35496
35497
35498
35499
35500
35501
35502
35503
35504
35505
35506
35507
35508
35509
35510
35511
35512
35513
35514
35515
35516
35517
35518
35519
35520
35521
35522
35523
35524
35525
35526
35527
35528
35529
35530
35531
35532
35533
35534
35535
35536
35537
35538
35539
35540
35541
35542
35543
35544
35545
35546
35547
35548
35549
35550
35551
35552
35553
35554
35555
35556
35557
35558
35559
35560
35561
35562
35563
35564
35565
35566
35567
35568
35569
35570
35571
35572
35573
35574
35575
35576
35577
35578
35579
35580
35581
35582
35583
35584
35585
35586
35587
35588
35589
35590
35591
35592
35593
35594
35595
35596
35597
35598
35599
35600
35601
3560

In [134]:
emails = final_dataset['corps']

In [135]:
i = 0
while i< len(emails):
  emails[i] = ' '.join(emails[i])
  i+=1

In [136]:
emails

0       dear student ambassador lutt you make promotio...
1       english dear fellow student for four week mobi...
2       this confirmation password instagram account o...
3       michel legault sport sds envoy wednesday octob...
4       confirmation trip monday november hello you pl...
                              ...                        
9706    imagine explore build tedxwoman event tedxpaju...
9707    welcome welcome welcome lutt here procedure fo...
9708    hello the rucher creative offer velos scholars...
9709    techcrunch newsletter saturday april greg kump...
9710    hello follow request allow enjoy beautiful spi...
Name: corps, Length: 9711, dtype: object

In [137]:
# Initiate UMAP
umap_model = UMAP(n_neighbors=15, 
                  n_components=5, 
                  min_dist=0.0, 
                  metric='cosine', 
                  random_state=100)
# Initiate BERTopic
topic_model = BERTopic(umap_model=umap_model, language = "multilingual", calculate_probabilities=True, nr_topics=12)
# Run BERTopic model
topics, probabilities = topic_model.fit_transform(emails)

In [138]:
# Get the list of topics
topic_model.get_topic_info()

,Topic,Count,Name
0,-1,6744,-1_the_student_message_hello
1,0,414,0_eur_ticket_travel_reservation
2,1,337,1_message_attachment_possible_utt
3,2,302,2_linkedin_ireland_wilton_company
4,3,289,3_france_linkedin_job_ile
5,4,266,4_application_interest_position_team
6,5,235,5_lydia_account_bank_eur
7,6,221,6_linkedin_oscar_network_join
8,7,219,7_store_monoprix_offer_eur
9,8,219,8_message_sport_utt_possible


In [139]:
# Get top 10 terms for a topic
topic_model.get_topic(0)

[('eur', 0.06405274297007824),
 ('ticket', 0.05625741867342069),
 ('travel', 0.03841230796729635),
 ('reservation', 0.037488385036643646),
 ('trip', 0.03452877742043855),
 ('paris', 0.03367169548257722),
 ('sncf', 0.029820611037625365),
 ('train', 0.029568881079271307),
 ('rental', 0.029484715458172685),
 ('flight', 0.02770242241537355)]

In [140]:
# Visualize top topic keywords
topic_model.visualize_barchart(top_n_topics=12)

In [141]:
def is_list_of_zeros(lst):
    for element in lst:
        if element != 0:
            return False
    return True

In [142]:
# utiliser le lexique

for topic in range(12):
    j = 0
    w = [tuple[0] for tuple in topic_model.get_topic(topic)]
    print(w)
    word_occurence = []
    for cat in lexique:
        nb = 0
        for lex in lexique[cat]:
            if lex in w:
                nb = nb+1
        word_occurence.append(nb)

    cat_number = word_occurence.index(max(word_occurence))
    print(word_occurence)
    keys = list(lexique.keys())
    # Use the index to access the key
    key = keys[cat_number]   
    if is_list_of_zeros(word_occurence):
      print("CATEGORIE pour le Topic "+str(topic)+" : Autre")
    else:
      print("CATEGORIE pour le Topic "+str(topic)+" : ", key)

['eur', 'ticket', 'travel', 'reservation', 'trip', 'paris', 'sncf', 'train', 'rental', 'flight']
[0, 0, 2, 0, 1, 0, 0, 0, 0, 1]
CATEGORIE pour le Topic 0 :  Commandes et Tickets
['message', 'attachment', 'possible', 'utt', 'the', 'internet', 'contamination', 'damage', 'cause', 'immediately']
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
CATEGORIE pour le Topic 1 : Autre
['linkedin', 'ireland', 'wilton', 'company', 'unlimited', 'plaza', 'dublin', 'profile', 'place', 'logo']
[0, 0, 0, 0, 0, 1, 0, 0, 0, 0]
CATEGORIE pour le Topic 2 :  Réseaux Sociaux
['france', 'linkedin', 'job', 'ile', 'paris', 'wilton', 'company', 'unlimited', 'ireland', 'plaza']
[0, 0, 0, 0, 0, 0, 0, 1, 0, 0]
CATEGORIE pour le Topic 3 :  Candidatures job et milieu Professionnel
['application', 'interest', 'position', 'team', 'thank', 'review', 'baptiste', 'amazon', 'datum', 'mathematic']
[0, 0, 1, 1, 0, 0, 0, 3, 0, 0]
CATEGORIE pour le Topic 4 :  Candidatures job et milieu Professionnel
['lydia', 'account', 'bank', 'eur', 'boursorama

In [143]:
# Visualize term rank decrease
topic_model.visualize_term_rank()

In [144]:
# Visualize intertopic distance
topic_model.visualize_topics()

In [145]:
# Visualize connections between topics using hierachical clustering
topic_model.visualize_hierarchy(top_n_topics=12)

In [146]:
# Visualize similarity using heatmap
topic_model.visualize_heatmap()